### Imports

In [1]:
#Install once
#!pip install imbalanced-learn --user
#!pip install delayed --user

import pandas as pd
import numpy as np
from datetime import datetime
from zipfile import ZipFile
import pickle
from pathlib import Path
import pytz
import csv


from sklearn.preprocessing import LabelEncoder, StandardScaler, MultiLabelBinarizer


In [2]:
#Read orig android data
origdata = pd.read_csv('data/android_bids_us.csv')
origdata.head()


,bidid,utc_time,app_id,user_state,user_isp,device_maker,device_model,device_osv,device_height,device_width,marketplace,click
0,87b0108c-6e9f-4783-8b80-8dd5aa3529a8,1536444331894,com.OppanaGames.CarSim,TX,AT&T Wireless,NaN,NaN,7.0,720,1280,chartboost,0
1,9284441f-a6ea-4698-9017-86436b92d416,1536444363932,com.mobilityware.CrownSolitaire,TX,T-Mobile USA,NaN,NaN,7.0,1280,720,chartboost,0
2,7e4779c2-f757-4324-8c2a-138b558b5a54,1536444386908,com.kuttigames.tenkyurollball,LA,AT&T Wireless,NaN,NaN,7.1.1,720,1280,chartboost,0
3,b64ea208-38ca-42ac-895e-0842d7352cc3,1536444405023,com.trendactionfree.call_of_sniper_duty_frontl...,TN,Comcast Cable,NaN,NaN,6.0.1,720,1280,chartboost,0
4,fd0c07cb-31f6-408c-9315-1cb652e76abc,1536444443458,com.landslab.my3rdgrademathgames,NC,AT&T U-verse,NaN,NaN,7.1.1,1024,768,chartboost,0


# Analyze android dataset

We test each column in android data set and see what stays and what should go

In [3]:
dataTypes = origdata.dtypes
print(f'Total columns: {dataTypes.size} ||| Total rows: {origdata.count()}')
print(f'Data type of each column of Dataframe : ')
print(dataTypes)
# We may have NAN's in user_isp, device_maker, device_model

print('-----------------------------------')
print('Amount of NaN under utc_time', origdata['utc_time'].isnull().sum())
# all items in marketplace col equal 'chartboost'. we can remove col

#Print start time 
start_time = origdata['utc_time'].min()
end_time = origdata['utc_time'].max()
print('Start Time: ',  datetime.fromtimestamp(start_time/1000))
print('End Time: ',  datetime.fromtimestamp(end_time/1000))

print('-----------------------------------')

print('-----------------------------------')
print('Variance data of android dataset')
print(origdata.var(ddof=0))
print('-----------------------------------')


print('-----------------------------------')
print(origdata.marketplace.unique())
# all items in marketplace col equal 'chartboost'. we can remove col
print('-----------------------------------')

print('-----------------------------------')
print('app_id::')
print('Amount of NaN under app_id', origdata['app_id'].isnull().sum())
print(origdata.app_id.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('user_isp::')
print('Amount of NaN under user_isp', origdata['user_isp'].isnull().sum())
print(origdata.user_isp.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('user_state::')
print(origdata.user_state.value_counts())
#Consider use onehotencoding insted of laelencoding
print('-----------------------------------')

print('-----------------------------------')
print('device_maker::')
print(origdata.device_maker.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('device_model::')
print(origdata.device_model.value_counts())
print('-----------------------------------')



Total columns: 12 ||| Total rows: bidid            3148828
utc_time         3148828
app_id           3148828
user_state       3148828
user_isp         3148789
device_maker     2184824
device_model     2184970
device_osv       3148828
device_height    3148828
device_width     3148828
marketplace      3148828
click            3148828
dtype: int64
Data type of each column of Dataframe : 
bidid            object
utc_time          int64
app_id           object
user_state       object
user_isp         object
device_maker     object
device_model     object
device_osv       object
device_height     int64
device_width      int64
marketplace      object
click             int64
dtype: object
-----------------------------------
Amount of NaN under utc_time 0
Start Time:  2018-08-06 21:00:20.974000
End Time:  2018-11-19 14:24:53.941000
-----------------------------------
-----------------------------------
Variance data of android dataset


C:\Users\micha\AppData\Local\Temp/ipykernel_28756/2273857839.py:21: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(origdata.var(ddof=0))


utc_time         5.383583e+18
device_height    2.694544e+05
device_width     2.049012e+05
click            6.312312e-02
dtype: float64
-----------------------------------
-----------------------------------
['chartboost']
-----------------------------------
-----------------------------------
app_id::
Amount of NaN under app_id 0
com.banananadco.blackjack                                                            54134
com.kiloo.subwaysurf                                                                 53670
toy.blast.pop.cubes.puzzle                                                           49726
com.blackcircleapps.halloweenbingo                                                   49273
com.mobilityware.spider                                                              44715
                                                                                     ...  
com.survival.games.craft.captain.robot.america.avengers.infinity.war.strike.force        1
com.greatringtonessounds.whistl

# Read Time Zones dictionary file

In [4]:
# Read time zones to convert utc and shift time based on the correct GMT align.
time_zones_csv = {}

with open('data/time_zones.csv', mode='r') as inp:
    reader = csv.reader(inp)
    time_zones_csv = {rows[0]:rows[2] for rows in reader}

print(time_zones_csv)

{'AK': 'America/Anchorage', 'AL': 'America/Chicago', 'AR': 'America/Chicago', 'AZ': 'America/Phoenix', 'CA': 'America/Los_Angeles', 'CO': 'America/Denver', 'CT': 'America/New_York', 'DC': 'America/New_York', 'DE': 'America/New_York', 'FL': 'America/New_York', 'GA': 'America/New_York', 'HI': 'Pacific/Honolulu', 'IA': 'America/Chicago', 'ID': 'America/Boise', 'IL': 'America/Chicago', 'IN': 'America/Kentucky/Louisville', 'KS': 'America/Chicago', 'KY': 'America/Kentucky/Monticello', 'LA': 'America/Chicago', 'MA': 'America/New_York', 'MD': 'America/New_York', 'ME': 'America/New_York', 'MI': 'America/Detroit', 'MN': 'America/Chicago', 'MO': 'America/Chicago', 'MS': 'America/Chicago', 'MT': 'America/Denver', 'NC': 'America/New_York', 'ND': 'America/North_Dakota/New_Salem', 'NE': 'America/Chicago', 'NH': 'America/New_York', 'NJ': 'America/New_York', 'NM': 'America/Denver', 'NV': 'America/Los_Angeles', 'NY': 'America/New_York', 'OH': 'America/New_York', 'OK': 'America/Chicago', 'OR': 'America/L

# Prepare android dataset

In [5]:
#Rearrange dataframe
#clone orig dataframe
android_df = origdata.copy()

#-----------------------------

# 'utc_time' col will be removed in the last part after splitting the data

#android_df['date'] = android_df['utc_time'].apply(lambda x: datetime.fromtimestamp(x/1000))
#android_df = android_df.drop(['utc_time'], axis=1)


#-----------------------------


#-----------------------------
# Fix 'utc_time' based on 'user_state' column

def update_time_utc(time_in_mili, state):
    current_timezone = pytz.timezone(time_zones_csv[state])
    localized_timestamp = current_timezone.localize(datetime.fromtimestamp(time_in_mili / 1000))
    
    return int(localized_timestamp.timestamp() * 1000)
android_df['utc_time'] = android_df.apply(lambda x:update_time_utc( x.utc_time, x.user_state), axis=1)

#-----------------------------


#-----------------------------
# Manipulate time column to extract hour from time. 
def utc_to_local(utc_dt):
    return datetime.fromtimestamp(utc_dt/1000).hour

# Add 'hour' column.
android_df['hour'] = android_df['utc_time'].apply(lambda x: utc_to_local(x))

#-----------------------------


#Remove marketplace col -> all values are the same
android_df = android_df.drop(['marketplace'], axis=1)

#-----------------------------

#Remove duplicated bidid. Keep only single instance keep the one with click == 1
android_df = android_df[~((android_df['bidid'].duplicated(keep=False))&(android_df['click']==0))]
#and then remove 'bidid' column
android_df = android_df.drop(['bidid'], axis=1)

#-----------------------------

#-----------------------------
#Change device_maker NaN's to unknown manufacturerer
#android_df['device_maker'] = android_df['device_maker'].apply(lambda x: 'unknown' if x!=x else x)
android_df.drop('device_maker', axis = 1,inplace = True)

#Change device_model NaN's to unknown valid model
#android_df['device_model'] = android_df['device_model'].apply(lambda x: 'unknown' if x!=x else x)
android_df.drop('device_model', axis = 1,inplace = True)
#-----------------------------

#-----------------------------
# Remove device_osv column
android_df.drop('device_osv', axis = 1,inplace = True)
#-----------------------------

#-----------------------------
# Take size cols and transform into resolution
android_df['screen_resolution'] = android_df['device_height']*android_df['device_width']
android_df.drop(['device_width', 'device_height'], axis=1, inplace=True)

# And then do scaling to new col
scaler = StandardScaler()
android_df['screen_resolution'] = scaler.fit_transform(android_df[['screen_resolution']])
#-----------------------------

#-----------------------------
# LabelEncoder for user_state
#Consider use onehotencoding insted of laele ncoding
le1 = LabelEncoder()
android_df['user_state'] = le1.fit_transform(android_df['user_state'])
#-----------------------------


#-----------------------------
# user_isp
# LabelEncoder for user_isp

#Consider use onehotencoding insted of laele ncoding
#android_df['user_isp'] = android_df['user_isp'].fillna(method="ffill")
#le2 = LabelEncoder()
#android_df['user_isp'] = le2.fit_transform(android_df['user_isp'])
 
# In our opinion, this column has no meaning to the desicion if the user will  click.
android_df.drop('user_isp', axis = 1,inplace = True)
#-----------------------------


android_df

,utc_time,app_id,user_state,click,hour,screen_resolution
0,1536473131894,com.OppanaGames.CarSim,43,0,9,-0.530933
1,1536473163932,com.mobilityware.CrownSolitaire,43,0,9,-0.530933
2,1536473186908,com.kuttigames.tenkyurollball,18,0,9,-0.530933
3,1536473205023,com.trendactionfree.call_of_sniper_duty_frontl...,42,0,9,-0.530933
5,1536469687429,com.kick.trucks.manual.shift.driving,9,0,8,-0.530933
...,...,...,...,...,...,...
3148822,1541501243393,com.qre.Coast.Lifeguard.Beach.Rescue,43,0,12,-0.530933
3148823,1541501258853,com.altitude.rrsv,43,0,12,-0.530933
3148825,1541501316760,br.com.tapps.penguinevolution,24,0,12,-0.530933
3148826,1541501352347,air.com.FDGEntertainment.Lonewolf.gp,48,0,12,0.857330


In [6]:
print(f'Click percent in data: {android_df.click.sum()/android_df.shape[0]*100}')


Click percent in data: 7.259269146156809


# Extract play_apps data and first clean

In [7]:
# Get play_apps data


#Read orig play_apps data
#app_details_orig = ZipFile("data/play_apps.zip")
#app_file = 'play_apps/a008.com.fc2.blog.androidkaihatu.datecamera2'
#app = pickle.loads(app_details_orig.read(app_file))
#app


def zipToDataFrame(zipped, to_remove):
    rows = []
    
    for f in zipped.filelist:
        app_file = f.filename
        try:
            app = pickle.loads(zipped.read(app_file))
        except EOFError as e:
            continue
        except pickle.UnpicklingError as e:
            continue
        except Exception as e:
            return
            
        # Remove columns
        for col in to_remove:
            del app[col]
        rows.append(app)
    
    return pd.DataFrame(rows)


app_details = ZipFile("data/play_apps.zip")
 # remove all of this column, all are textual.
cols_to_remove = ['title','screenshots', 'icon', 'size',
                  'description', 'description_html', 
                  'recent_changes', 'developer', 'developer_id', 'developer_email', 'developer_url',
                  'developer_address', 'url', 'current_version', 'updated', 'required_android_version',
                  'iap_range', 'reviews']


orig_play_apps = zipToDataFrame(app_details, cols_to_remove)
orig_play_apps



,video,category,score,histogram,editors_choice,price,free,iap,installs,content_rating,interactive_elements,app_id,bids
0,None,[PHOTOGRAPHY],4.0,"{5: 226, 4: 94, 3: 63, 2: 10, 1: 42}",False,0,True,False,"100,000+",[Everyone],None,a008.com.fc2.blog.androidkaihatu.datecamera2,484
1,None,[TOOLS],4.6,"{5: 1182, 4: 82, 3: 48, 2: 14, 1: 106}",False,0,True,False,"50,000+",[Everyone],None,a201706011153.xsky.txvpn,53
2,None,[PRODUCTIVITY],4.6,"{5: 1135, 4: 99, 3: 31, 2: 15, 1: 82}",False,0,True,False,"50,000+",[Everyone],None,a201706021616.vpn.turbovpn,43
3,https://www.youtube.com/embed/QulTvgUvuJ0,[TOOLS],3.9,"{5: 9, 4: 3, 3: 3, 2: 1, 1: 2}",False,0,True,False,"1,000+",[Everyone],None,a201707.grmo.a8bit.jp.beautytimer,60
4,None,[TOOLS],4.2,"{5: 670, 4: 165, 3: 141, 2: 49, 1: 87}",False,0,True,False,"100,000+",[Everyone],None,a2x.studio.fast.charging.battery.supercharging,600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29458,https://www.youtube.com/embed/JGXTEQbKUPo,[GAME_ACTION],4.5,"{5: 1923299, 4: 291276, 3: 120807, 2: 45512, 1...",True,0,True,True,"50,000,000+","[Teen, Violence, Blood]",[Digital Purchases],zombie.survival.craft.z,34140
29459,None,[ENTERTAINMENT],3.9,"{5: 104, 4: 18, 3: 14, 2: 3, 1: 33}",False,0,True,False,"10,000+",[Everyone],None,zombi.maps.forminecraftpe,248
29460,None,[GAME_ARCADE],4.4,"{5: 1487, 4: 254, 3: 179, 2: 51, 1: 129}",False,0,True,False,"500,000+",[Everyone],None,zone.super.jungle.worldrun2018,134
29461,https://www.youtube.com/embed/9_bq-wQkuWI,[GAME_PUZZLE],4.6,"{5: 20109, 4: 4404, 3: 1566, 2: 369, 1: 496}",False,0,True,True,"1,000,000+",[Everyone],[Digital Purchases],zoo.rescue,1137


# Analyze play_apps dataset

In [8]:
# explore play_apps data characteristics

apps_dataTypes = orig_play_apps.dtypes

print(f'Total columns: {apps_dataTypes.size} ||| Total rows: {orig_play_apps.count()}')
print(f'Data type of each column of Dataframe : ')
print(apps_dataTypes)
# We may have NAN's in iap_range, interactive_elements, price

print('-----------------------------------')
print('Amount of NaN under price', orig_play_apps['price'].isnull().sum())
print('Amount of NaN under free', orig_play_apps['free'].isnull().sum())
print('price values: ', orig_play_apps.price.unique())
print('free values: ',orig_play_apps.free.unique())

print('How many free from all dataset: ', (orig_play_apps['free'] == True).sum())
# items in price and free should be in a single col
# we can take the free col since the non free item is very little : about 1000 out of 29000
# or we can take col price since 0 represnts free
print('-----------------------------------')

print('-----------------------------------')
print('Variance data of play aps dataset')
print(orig_play_apps.var(ddof=0))
print('-----------------------------------')

print('-----------------------------------')
# items in price and free should be in a single col
print('Amount of NaN under score', orig_play_apps['score'].isnull().sum())
print('score: ', orig_play_apps.score.unique())
print('-----------------------------------')


'''print('-----------------------------------')
# Test reviews col - need to be scaled
print('Amount of NaN under reviews', orig_play_apps['reviews'].isnull().sum())
print('reviews: ', orig_play_apps.reviews.unique())
print('-----------------------------------')'''

print('-----------------------------------')
# Removed for testing
# Test bids col - need to be scaled
print('Amount of NaN under bids', orig_play_apps['bids'].isnull().sum())
print('bids: ', orig_play_apps.bids.unique())
print('-----------------------------------')

print('-----------------------------------')
# Removed for testing
print('content_rating column::: ')
# Test content_rating col
rating = orig_play_apps['content_rating'].agg(lambda x: ':'.join(map(str, x)))
print(rating.unique())
# We might drop this column since its require a lot of handling and im not sure its gonna impact our model
print('-----------------------------------')


print('-----------------------------------')
print('category column::: ')
# Test category col
category = orig_play_apps['category'].agg(lambda x: ':'.join(map(str, x)))
print(category.unique())
# We might drop this column since its require a lot of handling and im not sure its gonna impact our model
print('-----------------------------------')

print('-----------------------------------')
print('interactive_elements column::: ')
# Test category col
print('Amount of NaN under interactive_elements', orig_play_apps['interactive_elements'].isnull().sum())
interactive = orig_play_apps['interactive_elements'].agg(lambda x: ':'.join(map(str, x)))
interactive
#print(interactive.unique())
# We might drop this column since its require a lot of handling and im not sure its gonna impact our model
print('-----------------------------------')


Total columns: 13 ||| Total rows: video                    9150
category                29463
score                   29351
histogram               29463
editors_choice          29463
price                   28312
free                    29463
iap                     29463
installs                29462
content_rating          29463
interactive_elements    11271
app_id                  29463
bids                    29463
dtype: int64
Data type of each column of Dataframe : 
video                   object
category                object
score                   object
histogram               object
editors_choice            bool
price                   object
free                      bool
iap                       bool
installs                object
content_rating          object
interactive_elements    object
app_id                  object
bids                     int64
dtype: object
-----------------------------------
Amount of NaN under price 1151
Amount of NaN under free 0
price value

C:\Users\micha\AppData\Local\Temp/ipykernel_28756/1805933333.py:24: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(orig_play_apps.var(ddof=0))


# Prepare play_apps dataset

In [9]:
#Fix and manipulate play_apps data before merge with android_bids_us
#clone orig dataframe
play_apps_df = orig_play_apps.copy()

# Convert String to int.
def convert_installs(str_val=''):
    str_val = str_val.translate({ord(i):None for i in ',+'})
    return int(str_val)

# Convert String to float.
def convert_price(str_val=''):
    str_val = str_val.translate({ord(i):None for i in '$'})
    return float(str_val)

# Update mean value of 'score column'.
def update_mean(five, four, three, two, one):
    votes = five + four + three + two + one
    sum_column = (five * 5) + (four * 4) + (three * 3) + (two * 2) + one
    return sum_column / votes

#-------------------------------------------------
# Split histogram dictionary to separate columns.

play_apps_df = pd.concat([play_apps_df, play_apps_df['histogram'].apply(pd.Series)], axis=1)
play_apps_df.drop('histogram', axis = 1, inplace = True)

# Fill missing values from new columns.
for col in range(1, 6):
    play_apps_df[col] = play_apps_df[col].fillna(method="ffill")
    
# Because we changed the relation between 'score' and histogram we need to fix it.
play_apps_df['score'] = play_apps_df.apply(lambda row: update_mean(row[5], row[4], row[3], row[2], row[1]), axis = 1)
#-------------------------------------------------

#---------------------------------------------------
# Handle 'Installs' column
# Translate to number and then scale it
play_apps_df['installs'].fillna(method='pad', inplace=True)
play_apps_df['installs'] = play_apps_df['installs'].apply(lambda s: convert_installs(s))

# And then do scaling to new col
scaler = StandardScaler()
play_apps_df['installs'] = scaler.fit_transform(play_apps_df[['installs']])
#---------------------------------------------------


#---------------------------------------------------
# Handle 'video' column
play_apps_df['video'] = play_apps_df['video'].apply(lambda x: 0 if pd.isna(x) else 1)
#---------------------------------------------------

#---------------------------------------------------
# Handle 'score ' column
# Translate to number
play_apps_df['score'] = pd.to_numeric(play_apps_df['score'],errors = 'coerce')
play_apps_df['score'].fillna(method='pad', inplace=True)
play_apps_df['score'] = scaler.fit_transform(play_apps_df[['score']])
#---------------------------------------------------


# Reviews has been removed because it is the sum of histogram columns.
'''#---------------------------------------------------
# Handle 'reviews' column - we cant know weither the review is good or bad, 
# but the amount of reviews can point on popularity off the app
# There are no NaN's
# Translate to number
scaler = StandardScaler()
play_apps_df['reviews'] = scaler.fit_transform(play_apps_df[['reviews']])
#---------------------------------------------------'''


#---------------------------------------------------
# Price and free can be merged into single col named price.
# Convert free apps price to 0.
# Fill missing non-free applications with mean value if price.

# Keep rows that are not free and has no price.
rows_with_missing_price = play_apps_df[(play_apps_df['free'] == False) & (play_apps_df['price'].isna())]
# Remove from data frame.
play_apps_df.drop(rows_with_missing_price.index, inplace = True)

# Fill 'free' rows with 0 'price' value as string.
play_apps_df['price'] = play_apps_df['price'].fillna('0')
# Convert string price to a number.
play_apps_df['price'] = play_apps_df['price'].apply(lambda s: convert_price(s))

# Calculate mean price of non-free applications.
mean_price = play_apps_df[play_apps_df['price'] > 0]['price'].mean()

# Add the rows back.
play_apps_df = pd.concat([play_apps_df, rows_with_missing_price])

# Fill missing non-free prices with mean value.
play_apps_df['price'] = play_apps_df['price'].fillna(mean_price)
# Remove 'free' column.
play_apps_df.drop(['free'], axis=1, inplace=True)
#---------------------------------------------------


#---------------------------------------------------
# 'category' column transform to multil-lable
# Removed because it add a lot of time for testing and it insert noise to the data.
'''mlb = MultiLabelBinarizer()
mlb.fit(play_apps_df['category'])
new_col_names = mlb.classes_

# Create new DataFrame with transformed/one-hot encoded categories
categories = pd.DataFrame(mlb.fit_transform(play_apps_df['category']), columns=new_col_names)

# Concat with original `category` column
play_apps_df = pd.concat([play_apps_df, categories], axis=1 )
play_apps_df.drop('category', axis=1,inplace=True)'''
#---------------------------------------------------


#Temporary removed columns
#play_apps_df.drop(['bids'], axis=1, inplace=True)
play_apps_df.drop(['content_rating'], axis=1, inplace=True)
play_apps_df.drop(['interactive_elements'], axis=1, inplace=True)
play_apps_df.drop('category', axis=1,inplace=True)


play_apps_df

,video,score,editors_choice,price,iap,installs,app_id,bids,5,4,3,2,1
0,0,-0.389718,False,0.000000,False,-0.136503,a008.com.fc2.blog.androidkaihatu.datecamera2,484,226.0,94.0,63.0,10.0,42.0
1,0,0.872185,False,0.000000,False,-0.139379,a201706011153.xsky.txvpn,53,1182.0,82.0,48.0,14.0,106.0
2,0,1.014495,False,0.000000,False,-0.139379,a201706021616.vpn.turbovpn,43,1135.0,99.0,31.0,15.0,82.0
3,1,-0.760468,False,0.000000,False,-0.142198,a201707.grmo.a8bit.jp.beautytimer,60,9.0,3.0,3.0,1.0,2.0
4,0,-0.108808,False,0.000000,False,-0.136503,a2x.studio.fast.charging.battery.supercharging,600,670.0,165.0,141.0,49.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29018,0,-1.799229,False,2.638793,False,-0.141681,tsuyogoro.sugorokuon,194,55.0,24.0,22.0,12.0,28.0
29021,0,-0.924918,False,2.638793,False,-0.113491,tv.aberta,299,2505.0,614.0,516.0,286.0,749.0
29354,0,-1.177881,False,2.638793,False,-0.139379,www.sample.jp.sample_android,443,134.0,122.0,66.0,24.0,43.0
29378,0,-0.153552,False,2.638793,False,-0.141681,xyz.buildman.prizegame,1617,152.0,66.0,34.0,10.0,20.0


# Merge two datasets

In [10]:
print('---------------------------------------------')
#merge two data sets into merged dataframe
merged_df = pd.merge(android_df, play_apps_df, on='app_id')
merged_df = merged_df.drop(['app_id'], axis=1)

print('Rows before merge: ',android_df.shape[0])
print('Rows after merge: ', merged_df.shape[0])

#Concolusion after merge
# from android = 2936921 rows × 10 columns turn into 2656577 rows × 24 columns
# we missed ~300k rows. need to check y.....
print('---------------------------------------------')


merged_df




---------------------------------------------
Rows before merge:  2936921
Rows after merge:  2656577
---------------------------------------------


,utc_time,user_state,click,hour,screen_resolution,video,score,editors_choice,price,iap,installs,bids,5,4,3,2,1
0,1536473131894,43,0,9,-0.530933,1,0.323989,False,0.0,True,0.433052,10460,112010.0,15502.0,9283.0,3189.0,15145.0
1,1536487460352,14,0,13,-0.530933,1,0.323989,False,0.0,True,0.433052,10460,112010.0,15502.0,9283.0,3189.0,15145.0
2,1536483857259,9,0,12,3.294503,1,0.323989,False,0.0,True,0.433052,10460,112010.0,15502.0,9283.0,3189.0,15145.0
3,1535742138954,9,0,22,-0.530933,1,0.323989,False,0.0,True,0.433052,10460,112010.0,15502.0,9283.0,3189.0,15145.0
4,1536483732674,9,0,12,3.294503,1,0.323989,False,0.0,True,0.433052,10460,112010.0,15502.0,9283.0,3189.0,15145.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2656572,1534411090972,13,0,12,-1.147553,1,0.640467,False,0.0,True,-0.084725,3500,8260.0,719.0,529.0,141.0,881.0
2656573,1541488126798,24,0,9,-0.530933,0,-0.486189,False,0.0,True,-0.142198,36,7.0,0.0,1.0,0.0,2.0
2656574,1534334572980,12,0,15,0.857330,0,0.578545,False,0.0,True,-0.136503,48,281.0,72.0,39.0,6.0,17.0
2656575,1537496241561,9,0,5,-0.530933,0,1.426264,False,0.0,False,0.433052,3371,341560.0,26084.0,7907.0,2001.0,9841.0


# Split data and save to files

In [11]:
print('---------------------------------------------')
print('Amount of NaN under utc_time: ', merged_df['utc_time'].isnull().sum())
print('Amount of NaN in all data : ', merged_df.isnull().sum().sum())
print('---------------------------------------------')


print('---------------------------------------------')
#Print start time after merge
merged_start_time = merged_df['utc_time'].min()
merged_end_time = merged_df['utc_time'].max()
print('Start Time: ',  datetime.fromtimestamp(merged_start_time/1000))
print('End Time: ',  datetime.fromtimestamp(merged_end_time/1000))
print('Split time: ',  datetime.fromtimestamp(1541023200000/1000))
print('---------------------------------------------')


print('---------------------------------------------')
tarin_df, test_df = merged_df[(mask:=merged_df['utc_time'] < 1541023200000)], merged_df[~mask]

tarin_df = tarin_df.drop(['utc_time'], axis=1)
test_df = test_df.drop(['utc_time'], axis=1)

tarin_size = tarin_df.shape[0]
test_size = test_df.shape[0]

print('Amount of tarin_df data : ', tarin_size)
print('Amount of test_df data : ', test_size)

print(f'Test data is: {(test_size / (tarin_size + test_size))*100} percent')

      
print('---------------------------------------------')
      
#Save to CSV files
#Save to splited csv files
filepath1 = Path('data/train.csv')  
filepath2 = Path('data/test.csv')  
tarin_df.to_csv(filepath1, index=False)
test_df.to_csv(filepath2, index=False)



---------------------------------------------
Amount of NaN under utc_time:  0
Amount of NaN in all data :  0
---------------------------------------------
---------------------------------------------
Start Time:  2018-08-07 04:00:27.357000
End Time:  2018-11-20 00:23:09.706000
Split time:  2018-11-01 00:00:00
---------------------------------------------
---------------------------------------------
Amount of tarin_df data :  2287166
Amount of test_df data :  369411
Test data is: 13.905525795036244 percent
---------------------------------------------


# SMOTE balancing

After testing of the data we have tried to balance it using SMOTE.
It didn't work well becuase it insert noise.

In [ ]:
# Only if not installed
#!pip install imbalanced-learn
#!pip install delayed

In [ ]:
# Trying to eliminate not balanced clicks
from imblearn.over_sampling import SMOTE

oversample_smote = SMOTE()
X_smote, y_smote = oversample_smote.fit_resample(merged_df.drop('click', axis=1), merged_df['click'])

y_smote

In [ ]:
# Creating a new DataFrame from SMOTE

merged_smote_df = X_smote.copy()
merged_smote_df['click'] =  y_smote
train_smote_df, test_smote_df = merged_df[(mask:=merged_smote_df['utc_time'] < 1541023200000)], merged_smote_df[~mask]

train_smote_df = train_smote_df.drop(['utc_time'], axis=1)
test_smote_df = test_smote_df.drop(['utc_time'], axis=1)

print('Amount of train SMOTE_df data : ', train_smote_df.shape[0])
print('Amount of test SMOTE data : ', test_smote_df.shape[0])

print('---------------------------------------------')
      
#Save to CSV files
#Save to splited csv files
filepath1 = Path('data/train_SMOTE.csv')  
filepath2 = Path('data/test_SMOTE.csv')  
train_smote_df.to_csv(filepath1, index=False)
test_smote_df.to_csv(filepath2, index=False)
